# Loss Function

### 1. 개념
- 모델이 정답과 얼마나 다른지(오차)를 측정하는 함수
- 딥러닝 학습의 핵심 목표는 이 Loss를 최고화하는 것

### 2. 주요 손실 함수와 수식

| 함수 | 수식 | 주요 특징 | 사용 상황 | 장점 | 단점 |
|------|------|-----------|-----------|------|------|
| **MSE** (Mean Squared Error) | $MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$ | 예측과 실제값의 제곱 오차 | 회귀 문제 | 계산이 직관적이고 미분이 간단함 | 이상치에 민감함 |
| **MAE** (Mean Absolute Error) | $MAE = (abs) | 예측과 실제값의 절댓값 오차 | 회귀 문제 | 이상치에 덜 민감함 | 절댓값 함수는 미분 불연속 |
| **Cross Entropy** | $CE = - \sum y_i \log(\hat{y}_i)$ | 정답 확률이 높을수록 loss 감소 | 다중 분류 | 확률 기반 분류에 적합 | 확률이 0에 가까우면 수치 불안정 |
| **BCE** (Binary Cross Entropy) | $BCE = - [y \log(\hat{y}) + (1 - y) \log(1 - \hat{y})]$ | 이진 분류 전용 Cross Entropy | 이진 분류 | 확률 출력 해석 가능 | 정반대 예측 시 큰 페널티 |
| **KL Divergence** | $D_{KL}(P \| Q) = \sum_x P(x) \log \frac{P(x)}{Q(x)}$ | 두 확률 분포 간 차이 | VAE, 분포 비교 | 분포 자체를 학습 가능 | $Q(x) = 0$이면 발산 위험 |

### 3. 정보 이론
- Entropy : 정답 분포 자체의 정보량 (불확실할 때 정보량이 많음)
    - 평균 놀람도(정보량)라고도 불리며, "확률이 낮은 사건일수록, 일어났을 때 놀람이 큼"을 기반으로 함
    - 정보량 (surprise): $I(x) = -\log P(x)$ (확률에 반비례)
    - 엔트로피 (평균 정보량): $H(P) = \sum_x P(x) \cdot (-\log P(x)) = -\sum_x P(x) \log P(x)$
- Cross Entropy : 모델 예측 기준으로 봤을 때, 실제 정답에 필요한 정보량
    - 정답 분포 $P(x)$ 를 기준으로 사건별 예측한 정보량 $-\log Q(x)$ 를 평균낸 것
    - 정답 분포 $P$, 모델 예측 분포 $Q$일 때, $H(P, Q) = -\sum_x P(x) \log Q(x)$
    - 의미 : 모델이 예측한 분포 $Q$를 기준으로, 정답 $P$를 설명하려면 **얼마나 많은 정보(코스트)가 필요한가?** -> "LOSS"
- KL Divergence : 예측한 확률 분포가 정답 분포에서 얼마나 벗어났는지 (=정보 낭비량)
    - $D_{KL}(P \| Q)$ = $H(P, Q) - H(P)$ = $\sum_x P(x) \log \frac{P(x)}{Q(x)}$
    - 예측이 완벽하면 $Q = P$ 이므로 $D_{KL} = 0$
    - 분포를 맞추는 문제, 예: VAE에서 활용

In [ ]:
import torch
import torch.nn as nn

y_pred = torch.tensor([2.5, 0.0, 2.1])
y_true = torch.tensor([3.0, -0.5, 2.0])

loss_fn = nn.MSELoss()  # nn.L1Loss() (MAE), nn.BCELoss()
loss = loss_fn(y_pred, y_true)
print("MSE Loss:", loss.item())

MSE Loss: 0.17000000178813934


In [15]:
logits = torch.tensor([[2.0, 1.0, 0.1]])
labels = torch.tensor([0])  # 정답 클래스 인덱스 -> 다중 클래스 분류

loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(logits, labels)  # CrossEntropyLoss 내부에서 Softmax & log 자동 계산 
print("Cross Entropy Loss:", loss.item())

Cross Entropy Loss: 0.4170299470424652


In [8]:
import torch.nn.functional as F

P = torch.tensor([0.5, 0.5])  # 정답 분포
Q = torch.tensor([0.9, 0.1])  # 예측 분포

log_Q = Q.log()
kl = F.kl_div(log_Q, P, reduction='batchmean')
print("KL Divergence:", kl.item())


KL Divergence: 0.2554128170013428
